Create a conda environment:

<code> conda create --name dlPyTorch </code>

Activate the conda environment:

<code> conda activate dlPyTorch </code>

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.rand(2,2))

True
tensor([[0.6582, 0.8258],
        [0.2301, 0.3184]])


<code> torch.cuda.is_available() </code> 

Here, the above line should return **TRUE** : PyTorch detects Graphics card. 
                                   **FALSE**: Need to reverify the Cuda installation.

## Tensors

A simple scalar (e.g., 1) can be represented as a tensor of rank 0, a vector is rank 1, 
an n × n matrix is rank 2, and so on. In the previous example, we created a rank 2
tensor with random values by using torch.rand().

In [3]:
x = torch.tensor([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
x

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])

We can change an element in a tensor by using standard Python indexing:

In [4]:
x[0][3]=9
x

tensor([[1, 0, 0, 9],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])

A special creation of functions to generate particular types of tensors. In particular, ones() and zeroes() will generate tensors filled
with 1s and 0s, respectively:

In [5]:
y=torch.zeros(4,4)
z=torch.ones(4,4)

print(y)
z

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

We can do some mathematical operations of tensors, shown below:

In [6]:
print(y+z)
print(x+z)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[ 2.,  1.,  1., 10.],
        [ 1.,  2.,  1.,  1.],
        [ 1.,  1.,  2.,  1.],
        [ 1.,  1.,  1.,  2.]])


And if you have a tensor of rank 0 (a scalar), you can pull out the value with ***item()***:

In [7]:
torch.rand(1).item()

0.22166800498962402

Tensors can live in the CPU or on the GPU and can be copied between
devices by using the to() function:

In [8]:
cpu_tensor = x
cpu_tensor.device

device(type='cpu')

In [9]:
gpu_tensor = cpu_tensor.to("cuda")
gpu_tensor.device

device(type='cuda', index=0)

First, we often need to find the maximum item in a tensor as well as the
index that contains the maximum value (as this often corresponds to the
class that the neural network has decided upon in its final prediction). These
can be done with the ***max()*** and ***argmax()*** functions. We can also use
item() to extract a standard Python value from a 1D tensor.

In [10]:
p=torch.rand(3,3)
print(p)
print(p.max())
p.max().item()

tensor([[0.4062, 0.1836, 0.3082],
        [0.8257, 0.2483, 0.1372],
        [0.4104, 0.8930, 0.2575]])
tensor(0.8930)


0.8929583430290222

To change the type of a tensor; for example, from a
LongTensor to a FloatTensor. We can do this with ***to()***:

In [11]:
x.type()

'torch.LongTensor'

In [12]:
m=torch.tensor([[1,2],[3,4]]).to(dtype=torch.float32)
print(m)
m.type()

tensor([[1., 2.],
        [3., 4.]])


'torch.FloatTensor'

In PyTorch, functions that modify tensors come in two forms:

1. Out-of-place (regular) functions
These create a new tensor while keeping the original tensor unchanged.

In [13]:
random_tensor = torch.rand(2,2)
new_tensor = random_tensor.log2()  # Creates a new tensor
print(random_tensor)  # Original tensor remains unchanged
print(new_tensor)  # New tensor with log2 values


tensor([[0.1387, 0.0892],
        [0.6188, 0.9524]])
tensor([[-2.8505, -3.4870],
        [-0.6924, -0.0703]])


2. In-place functions (with _ suffix)
These modify the original tensor directly to save memory.

In [14]:
random_tensor.log2_()  # Modifies random_tensor in-place
print(random_tensor)  # Now holds log2 values

tensor([[-2.8505, -3.4870],
        [-0.6924, -0.0703]])


Another common operation is reshaping a tensor. This can often occur
because your neural network layer may require a slightly different input
shape than what you currently have to feed into it. 
For example, the
Modified National Institute of Standards and Technology (MNIST) dataset
of handwritten digits is a collection of 28 × 28 images, but the way it’s
packaged is in arrays of length 784. To use the networks we are
constructing, we need to turn those back into 1 × 28 × 28 tensors (the
leading 1 is the number of channels—normally red, green, and blue—but as
MNIST digits are just grayscale, we have only one channel). We can do this
with either ***view()*** or ***reshape()***:

In [15]:
flat_tensor = torch.rand(784)
viewed_tensor = flat_tensor.view(1,28,28)
viewed_tensor.shape

torch.Size([1, 28, 28])

In [16]:
flat_tensor.shape

torch.Size([784])

In [17]:
reshaped_tensor = flat_tensor.reshape(1,28,28)
reshaped_tensor.shape

torch.Size([1, 28, 28])

Finally, you might need to rearrange the dimensions of a tensor. You will
likely come across this with images, which often are stored as **[height,
width, channel]** tensors, but PyTorch prefers to deal with these in a
**[channel, height, width]**. You can user permute() to deal with these
in a fairly straightforward manner:

In [18]:
hwc_tensor = torch.rand(640, 480, 3)
chw_tensor = hwc_tensor.permute(2,0,1)
chw_tensor.shape

torch.Size([3, 640, 480])

Here, we’ve just applied permute to a **[640,480,3]** tensor, with the
arguments being the indexes of the tensor’s dimensions, so we want the
final dimension (2, due to zero indexing) to be at the front of our tensor,
followed by the remaining two dimensions in their original order.

# Tensor Broadcasting

Borrowed from NumPy, broadcasting allows you to perform operations
between a tensor and a smaller tensor. You can broadcast across two tensors
if, starting backward from their trailing dimensions:

1. The two dimensions are equal.

2. One of the dimensions is 1.

In our use of broadcasting, it works because 1 has a dimension of 1, and as
there are no other dimensions, the 1 can be expanded to cover the other
tensor. If we tried to add a **[2,2]** tensor to a **[3,3]** tensor, we’d get this
error message:
`The size of tensor a (2) must match the size of
tensor b (3) at non-singleton dimension 1`
But we could add a **[1,3]** tensor to the **[3,3]** tensor without any trouble.
Broadcasting is a handy little feature that increases brevity of code, and is
often faster than manually expanding the tensor yourself.




Broadcasting Rules
To perform an operation between two tensors, PyTorch applies broadcasting if their shapes do not match exactly. The rules are:

Trailing dimensions must either be equal or one of them must be 1.

Missing dimensions in the smaller tensor are considered as 1.

In [19]:
A = torch.tensor([[1, 2, 3]])  # Shape: [1, 3]
B = torch.tensor([[4], [5], [6]])  # Shape: [3, 1]

C = A + B  # Broadcasting happens here
print(C)


tensor([[5, 6, 7],
        [6, 7, 8],
        [7, 8, 9]])


In [21]:
A = torch.rand(2, 2)  # Shape: [2, 2]
B = torch.rand(3, 3)  # Shape: [3, 3]

C = A + B  # RuntimeError: The size of tensor A (2) must match the size of tensor B (3)  

           #[2, 2] cannot be broadcasted to [3, 3] because neither dimension is 1.


RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

In [22]:
A = torch.rand(1, 3)  # Shape: [1, 3]
B = torch.rand(3, 3)  # Shape: [3, 3]

C = A + B  # Works! Broadcasting expands A to [3, 3]


A expands to [3, 3]

B remains [3, 3]

Element-wise addition happens successfully.

## Why use broadcasting?

1. Efficient Computation
2. Saves Memory
3. Concise code

Till now, We have introduced the fundamental building block of the
library, the tensor. In the next notebook, to start building neural networks and classifying images, so one should be comfortable with tensors and their operations.